# Packages

In [4]:
using Pkg; Pkg.activate("."); 
using StatsBase
using MUST
using PyPlot
using Interpolations
using PyCall
using KernelDensity
using DelimitedFiles
using Printf
using TSO

#import PyCall.rc
#rc("font", size=17)

MUST.@import_dispatch "/u/peitner/DISPATCH/dispatch2"

MUST.@get_help visual
pl = pyimport("matplotlib.pylab");

  Activating environment at `/cobra/ptmp/peitner/model_grid/MUST.jl/examples/solar_model/Project.toml`


In [5]:
plt.style.use("dark_background")

# Model

In [6]:
folders = [ MUST.@in_dispatch("data_tmps/DIS_MARCS_v0.4.4_TDZ2")]
snaps   = [93]
labels  = ["MARCS (4 bins) - cooler adiabat"]

model, model_t = [first(MUST.Boxes(folder1, snaps=[snaps[i]])) for (i,folder1) in enumerate(folders)], 
                 [last(MUST.Boxes(folder1, snaps=[snaps[i]])) for (i,folder1) in enumerate(folders)];

# 3D surface plot

In [7]:
m = first(model);

In [8]:
mesh(m::MUST.Box) = MUST.meshgrid(MUST.axis(m, :x) ./1e8, MUST.axis(m, :y) ./1e8, MUST.axis(m, :z) ./1e8)

mesh (generic function with 1 method)

In [16]:
b = first(MUST.Boxes(first(folders)))

m = b[1]

paths = []

# Define dimensions
Nx, Ny, Nz = size(m)
X, Y, Z    = mesh(m)

xmin, xmax = minimum(X), maximum(X)
ymin, ymax = minimum(Y), maximum(Y)
zmin, zmax = minimum(Z), maximum(Z)

vmin = 2000 
vmax = 15000

var = :T

for i in eachindex(b)
    #i > 5&& break
    m = b[i]

    close()

    # Define dimensions
    Nx, Ny, Nz = size(m)
    X, Y, Z    = mesh(m)

    #mask = (1:5, 1:5, 1:5)

    data = m[var]

    # Create a figure with 3D ax
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection="3d")

    im = ax.scatter(X[:,   1,  :],   Y[:,   1, :],   Z[:,   1, :],   c=data[:,   1, :],   s=6.5, vmin=vmin, vmax=vmax, cmap="rainbow")
    im = ax.scatter(X[end, :,  :],   Y[end, :, :],   Z[end, :, :],   c=data[end, :, :],   s=6.5, vmin=vmin, vmax=vmax, cmap="rainbow")
    im = ax.scatter(X[:,   :,  end], Y[:,   :, end], Z[:,   :, end], c=data[:,   :, end], s=6.5, vmin=vmin, vmax=vmax, cmap="rainbow")
    im = ax.scatter(X[:,   end, :],  Y[:, end, :],   Z[:, end, :],   c=data[:, end, :],   s=6.5, vmin=vmin, vmax=vmax, cmap="rainbow")
    im = ax.scatter(X[1,   :,  :],   Y[1,   :, :],   Z[1,   :, :],   c=data[1,   :, :],   s=6.5, vmin=vmin, vmax=vmax, cmap="rainbow")
    im = ax.scatter(X[:,   :,  1],   Y[:,   :, 1],   Z[:,   :, 1],   c=data[:,   :, 1],   s=6.5, vmin=vmin, vmax=vmax, cmap="rainbow")

    # Set limits of the plot from coord limits
    ax.set(xlim=[xmin, xmax], ylim=[ymin, ymax], zlim=[zmin, zmax])

    # Plot edges
    ax.plot([xmax, xmax], [ymin, ymax], zmax, color="0.4", linewidth=1, zorder=1e3)
    ax.plot([xmin, xmax], [ymin, ymin], zmax, color="0.4", linewidth=1, zorder=1e3)
    ax.plot([xmax, xmax], [ymin, ymax], zmax, color="0.4", linewidth=1, zorder=1e3)
    ax.plot([xmin, xmax], [ymin, ymin], zmax, color="0.4", linewidth=1, zorder=1e3)

    ax.plot([xmax, xmax], [ymin, ymax], 0, color="0.4", linewidth=1, zorder=1e3)
    ax.plot([xmin, xmax], [ymin, ymin], 0, color="0.4", linewidth=1, zorder=1e3)
    ax.plot([xmax, xmax], [ymin, ymax], 0, color="0.4", linewidth=1, zorder=1e3)
    ax.plot([xmin, xmax], [ymin, ymin], 0, color="0.4", linewidth=1, zorder=1e3)

    ax.plot([xmax, xmax], [ymin, ymin], [zmin, zmax], color="0.4", linewidth=1, zorder=1e3)

    # Set labels and zticks
    ax.set(
        ylabel="Y [Mm]",
        xlabel="X [Mm]",
        zlabel="Z [Mm]")

    # Set zoom and angle view
    #ax.view_init(40, -30, 0)
    #ax.set_box_aspect(None, zoom=0.9)

    norm = matplotlib.colors.Normalize(vmin=vmin, vmax=vmax)
    sm   = plt.cm.ScalarMappable(norm=norm, cmap="rainbow")
    sm.set_array([]) 

    # Colorbar
    plt.colorbar(sm, ax=ax, label="Temperature [K]")

    ax.grid(false)

    # make the panes transparent
    ax.xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax.yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
    ax.zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))

    ts = @sprintf("%i", b[i].parameter.time)
    ax.set_title("solar time: $(ts) min", fontsize="x-large")

    fig.savefig("mtemp_$(i).png", bbox_inches="tight", dpi=300, transparent=false)
    append!(paths, ["mtemp_$(i).png"])

    plt.close()
end

gif_from_pngs(paths, "cube_$(var).gif", duration=0.45);